# Common imports & library functions

In [4]:
import doctest
import math

# Day 1: The Tyranny of the Rocket Equation

In [5]:
def fuel_requirements(mass):
    """
    >>> fuel_requirements(12)
    2
    >>> fuel_requirements(14)
    2
    >>> fuel_requirements(1969)
    654
    >>> fuel_requirements(100756)
    33583
    """
    return mass // 3 - 2

def meta_fuel_requirements(mass):
    """
    >>> meta_fuel_requirements(14)
    2
    >>> meta_fuel_requirements(1969)
    966
    >>> meta_fuel_requirements(100756)
    50346
    """
    if (req := fuel_requirements(mass)) < 0:
        return 0
    else:
        return req + meta_fuel_requirements(req)


def total_fuel_requirements(masses):
    return sum(fuel_requirements(m) for m in masses)

def total_meta_fuel_requirements(masses):
    return sum(meta_fuel_requirements(m) for m in masses)

In [6]:
# Run unit tests
doctest.run_docstring_examples(fuel_requirements, globs=None, verbose=True)

Finding tests in NoName
Trying:
    fuel_requirements(12)
Expecting:
    2
ok
Trying:
    fuel_requirements(14)
Expecting:
    2
ok
Trying:
    fuel_requirements(1969)
Expecting:
    654
ok
Trying:
    fuel_requirements(100756)
Expecting:
    33583
ok


In [7]:
# Run unit tests
doctest.run_docstring_examples(meta_fuel_requirements, globs=None, verbose=True)

Finding tests in NoName
Trying:
    meta_fuel_requirements(14)
Expecting:
    2
ok
Trying:
    meta_fuel_requirements(1969)
Expecting:
    966
ok
Trying:
    meta_fuel_requirements(100756)
Expecting:
    50346
ok


In [8]:
# Final answers
with open('day1_input.txt') as f:
    values = [int(l.strip()) for l in f]
    print('Part 1: ', total_fuel_requirements(values))
    print('Part 2: ', total_meta_fuel_requirements(values))

Part 1:  3152375
Part 2:  4725720


# Day 2: 1202 Program Alarm

In [35]:
class Opcode:
    ADD = 1
    MULT = 2
    HALT = 99

def execute(program, pc=0):
    """
    >>> execute([1,0,0,0,99])
    [2, 0, 0, 0, 99]
    >>> execute([1,1,1,4,99,5,6,0,99])
    [30, 1, 1, 4, 2, 5, 6, 0, 99]
    """
    while pc < len(program):
        if program[pc] == Opcode.HALT:
            return program
        
        src_a, src_b, dst = program[pc+1:pc+4]
        if program[pc] == Opcode.ADD:
            program[dst] = program[src_a] + program[src_b]
        elif program[pc] == Opcode.MULT:
            program[dst] = program[src_a] * program[src_b]
        else:
            raise Exception('Invalid opcode: {}'.format(program[pc]))
        pc += 4
    raise Exception('Execution terminated without halt instruction')

In [36]:
# Run unit tests
doctest.run_docstring_examples(execute, globs=None, verbose=True)

Finding tests in NoName
Trying:
    execute([1,0,0,0,99])
Expecting:
    [2, 0, 0, 0, 99]
ok
Trying:
    execute([1,1,1,4,99,5,6,0,99])
Expecting:
    [30, 1, 1, 4, 2, 5, 6, 0, 99]
ok


In [44]:
# Final answers
with open('day2_input.txt') as f:
    values = [int(l) for l in f.read().split(',')]
    # Restore to 1202 state:
    values[1] = 12
    values[2] = 2
    print('Part 1: ', execute(values))

Part 1:  [5110675, 12, 2, 2, 1, 1, 2, 3, 1, 3, 4, 3, 1, 5, 0, 3, 2, 9, 1, 36, 1, 5, 19, 37, 2, 9, 23, 111, 1, 27, 5, 112, 2, 31, 13, 560, 1, 35, 9, 563, 1, 39, 10, 567, 2, 43, 9, 1701, 1, 47, 5, 1702, 2, 13, 51, 8510, 1, 9, 55, 8513, 1, 5, 59, 8514, 2, 6, 63, 17028, 1, 5, 67, 17029, 1, 6, 71, 17031, 2, 9, 75, 51093, 1, 79, 13, 51098, 1, 83, 13, 51103, 1, 87, 5, 51104, 1, 6, 91, 51106, 2, 95, 13, 255530, 2, 13, 99, 1277650, 1, 5, 103, 1277651, 1, 107, 10, 1277655, 1, 111, 13, 1277660, 1, 10, 115, 1277664, 1, 9, 119, 1277667, 2, 6, 123, 2555334, 1, 5, 127, 2555335, 2, 6, 131, 5110670, 1, 135, 2, 5110672, 1, 139, 9, 0, 99, 2, 14, 0, 0]
